<a href="https://colab.research.google.com/github/jeniferGoncalvesDaSilvaDev/maya_chat_bot/blob/main/Maya_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install nltk
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 55.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
# Instalar as bibliotecas necessárias (se estiver rodando em Colab)

# Importar as bibliotecas
from transformers import pipeline
import random
import nltk
import torch
from sentence_transformers import SentenceTransformer, util

# Baixar pacotes necessários para tokenização NLTK
nltk.download('punkt')

# Carregar o modelo de análise de sentimentos mais robusto
sentiment_analyzer = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment")

# Carregar o modelo de embeddings para análise semântica
modelo_embeddings = SentenceTransformer('all-MiniLM-L6-v2')

# Histórico de interações
historico_interacao = []

# Respostas baseadas no sentimento
respostas_positivas = [
    "Fico muito feliz em saber disso! Quer me contar mais?",
    "Isso é ótimo! O que mais te animou hoje?",
    "Adoro ouvir boas notícias! O que mais você gostaria de compartilhar?"
]

respostas_negativas = [
    "Sinto muito que esteja passando por isso. Quer conversar mais sobre o que está te incomodando?",
    "Lamento saber disso. Posso te ajudar de alguma forma?",
    "Estou aqui para ouvir. Me conta mais sobre o que está acontecendo."
]

respostas_neutras = [
    "Entendo. Pode me explicar um pouco mais?",
    "Interessante! Como você se sente em relação a isso?",
    "Isso faz sentido. Pode me dar mais detalhes?"
]

# Frases curtas e ambíguas para tratamento especial
frases_ambiguas = {
    "não": "Poderia me explicar melhor?",
    "sim": "Ótimo! Quer me contar mais?",
    "tanto faz": "Entendo, parece que você está neutro quanto a isso."
}

# Função para analisar o sentimento e responder
def analisar_sentimento_responder(texto_usuario):
    # Verifica se a frase está na lista de ambíguas
    if texto_usuario.lower() in frases_ambiguas:
        return frases_ambiguas[texto_usuario.lower()], "NEUTRAL", 1.0

    # Analisar sentimento
    analise = sentiment_analyzer(texto_usuario)[0]
    sentimento = analise['label']
    score = analise['score']

    # Mapear os labels do modelo para POSITIVE, NEGATIVE e NEUTRAL
    if sentimento == "LABEL_2":
        sentimento = "POSITIVE"
    elif sentimento == "LABEL_1":
        sentimento = "NEUTRAL"
    else:
        sentimento = "NEGATIVE"

    # Adicionar ao histórico de interações
    historico_interacao.append({"texto": texto_usuario, "sentimento": sentimento, "score": score})

    # Escolher resposta apropriada
    if sentimento == "POSITIVE":
        resposta = random.choice(respostas_positivas)
    elif sentimento == "NEGATIVE":
        resposta = random.choice(respostas_negativas)
    else:
        resposta = random.choice(respostas_neutras)

    return resposta, sentimento, score

# Função para prever sentimento após 10 interações
def prever_proximo_sentimento():
    if len(historico_interacao) < 10:
        return None  # Não prever antes de ter 10 interações

    sentimentos = [entry['sentimento'] for entry in historico_interacao[-10:]]
    positivos = sentimentos.count("POSITIVE")
    negativos = sentimentos.count("NEGATIVE")

    if positivos > negativos:
        return "Com base nas últimas conversas, parece que você está em um bom momento!"
    elif negativos > positivos:
        return "Notei que você expressou sentimentos negativos com frequência. Quer conversar mais sobre isso?"
    else:
        return "Seu humor está variando. Se precisar, estou aqui para ouvir."

# Função para encontrar respostas semelhantes na memória
def encontrar_resposta_semelhante(nova_entrada):
    if not historico_interacao:
        return None

    # Criar embedding da nova entrada
    embedding_nova_entrada = modelo_embeddings.encode(nova_entrada, convert_to_tensor=True)

    # Criar embeddings para interações anteriores
    embeddings_historico = [modelo_embeddings.encode(entry["texto"], convert_to_tensor=True) for entry in historico_interacao]

    # Calcular similaridade
    similaridades = [util.pytorch_cos_sim(embedding_nova_entrada, emb)[0][0].item() for emb in embeddings_historico]

    # Encontrar a interação mais semelhante
    indice_mais_similar = similaridades.index(max(similaridades))

    if similaridades[indice_mais_similar] > 0.7:
        return historico_interacao[indice_mais_similar]["texto"]

    return None

# Função principal de interação com a IA
def interagir_com_IA():
    print("Olá! Eu sou o Mayá. Como você está se sentindo hoje?")
    while True:
        entrada = input("Você: ")
        if entrada.lower() in ['sair', 'exit', 'parar', 'fim']:
            print("Mayá: Foi um prazer conversar com você. Estarei aqui sempre que quiser falar.")
            break

        # Verifica se já houve uma conversa semelhante antes
        resposta_semelhante = encontrar_resposta_semelhante(entrada)
        if resposta_semelhante:
            print(f"Mayá: Você já mencionou algo parecido antes: '{resposta_semelhante}'. Quer me contar mais sobre isso?")
        else:
            resposta, sentimento, score = analisar_sentimento_responder(entrada)
            print(f"Mayá ({sentimento}, confiança: {score:.2f}): {resposta}")

        # Prever sentimento após 10 interações
        previsao = prever_proximo_sentimento()
        if previsao:
            print("Mayá (Análise preditiva):", previsao)

        print("-" * 50)

# Rodar a IA
interagir_com_IA()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/747 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Olá! Eu sou o Mayá. Como você está se sentindo hoje?
Você: sair
Mayá: Foi um prazer conversar com você. Estarei aqui sempre que quiser falar.


In [ ]:
pip install streamlit transformers nltk sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 63.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.3 MB/s eta 0:00:00


In [ ]:
import streamlit as st
from transformers import pipeline
import random
import nltk
from sentence_transformers import SentenceTransformer, util

# Baixar pacotes necessários para tokenização NLTK
nltk.download('punkt')

# Carregar o modelo de análise de sentimentos
sentiment_analyzer = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment")

# Carregar o modelo de embeddings para análise semântica
modelo_embeddings = SentenceTransformer('all-MiniLM-L6-v2')

# Histórico de interações
historico_interacao = []

# Respostas baseadas no sentimento
respostas_positivas = [
    "Fico muito feliz em saber disso! Quer me contar mais?",
    "Isso é ótimo! O que mais te animou hoje?",
    "Adoro ouvir boas notícias! O que mais você gostaria de compartilhar?"
]

respostas_negativas = [
    "Sinto muito que esteja passando por isso. Quer conversar mais sobre o que está te incomodando?",
    "Lamento saber disso. Posso te ajudar de alguma forma?",
    "Estou aqui para ouvir. Me conta mais sobre o que está acontecendo."
]

respostas_neutras = [
    "Entendo. Pode me explicar um pouco mais?",
    "Interessante! Como você se sente em relação a isso?",
    "Isso faz sentido. Pode me dar mais detalhes?"
]

# Função para analisar o sentimento e responder
def analisar_sentimento_responder(texto_usuario):
    analise = sentiment_analyzer(texto_usuario)[0]
    sentimento = analise['label']
    score = analise['score']

    if sentimento == "LABEL_2":
        sentimento = "POSITIVE"
    elif sentimento == "LABEL_1":
        sentimento = "NEUTRAL"
    else:
        sentimento = "NEGATIVE"

    # Adicionar ao histórico de interações
    historico_interacao.append({"texto": texto_usuario, "sentimento": sentimento, "score": score})

    # Escolher resposta apropriada
    if sentimento == "POSITIVE":
        resposta = random.choice(respostas_positivas)
    elif sentimento == "NEGATIVE":
        resposta = random.choice(respostas_negativas)
    else:
        resposta = random.choice(respostas_neutras)

    return resposta, sentimento, score

# Função para encontrar respostas semelhantes na memória
def encontrar_resposta_semelhante(nova_entrada):
    if not historico_interacao:
        return None

    # Criar embedding da nova entrada
    embedding_nova_entrada = modelo_embeddings.encode(nova_entrada, convert_to_tensor=True)

    # Criar embeddings para interações anteriores
    embeddings_historico = [modelo_embeddings.encode(entry["texto"], convert_to_tensor=True) for entry in historico_interacao]

    # Calcular similaridade
    similaridades = [util.pytorch_cos_sim(embedding_nova_entrada, emb)[0][0].item() for emb in embeddings_historico]

    # Encontrar a interação mais semelhante
    indice_mais_similar = similaridades.index(max(similaridades))

    if similaridades[indice_mais_similar] > 0.7:
        return historico_interacao[indice_mais_similar]["texto"]

    return None

# Título da aplicação Streamlit
st.title("Assistente de Sentimentos - Mayá")

# Entrada do usuário
texto_usuario = st.text_input("Como você está se sentindo hoje?")

if texto_usuario:
    # Verifica se já houve uma conversa semelhante antes
    resposta_semelhante = encontrar_resposta_semelhante(texto_usuario)
    if resposta_semelhante:
        st.write(f"Você já mencionou algo parecido antes: '{resposta_semelhante}'. Quer me contar mais sobre isso?")
    else:
        resposta, sentimento, score = analisar_sentimento_responder(texto_usuario)
        st.write(f"Mayá ({sentimento}, confiança: {score:.2f}): {resposta}")

    # Mostrar o histórico de interações
    if len(historico_interacao) > 0:
        st.subheader("Histórico de Interações:")
        for entry in historico_interacao:
            st.write(f"Texto: {entry['texto']} - Sentimento: {entry['sentimento']} (Confiança: {entry['score']:.2f})")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Device set to use cpu
2025-03-13 21:21:09.449 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-13 21:21:09.621 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-03-13 21:21:09.622 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-13 21:21:09.625 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-13 21:21:09.627 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-13 21:21:09.628 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-13 21:21:09.628 Thread 'MainThre